In [1]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

d:\Software\Anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [527]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp

In [2]:
df=pd.read_csv("r2.csv")

In [3]:
CPCleanedUnique=df.CPCleaned.unique()
        
def getOriginalQ(cleanedQ):
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

with open("grpdQ.txt","r") as txt:
    grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

In [63]:
def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def tab(cp):
    global df
    filtered_df=df[df['CPCleaned'] ==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()


In [9]:
undo('Storage of chemicals and hazardous substances')

In [13]:
# replace([
#     "prevent workers' exposure to hazardous substances?	",
#     "Has the employer taken actions to assess, monitor, prevent and/or limit workers' exposure to hazardous chemicals?",
#     "Has the employer taken actions to effectively assess, monitor, prevent and/or limit workers' exposure to hazardous chemicals?	"
# ])

# replace([
#     "Are chemicals and hazardous substances properly stored ?	",
#     "Are chemicals and hazardous substances properly stored?",
#     "Are chemicals and hazardous substances properly stored? [differentiation question]",
#     "Does the employer keep an inventory of hazardous chemicals used in the workplace?"
# ])

In [27]:
# replace(["Does the factory have a functioning Participation Committee?",
#          "Does dialogue at the workplace take place in compliance with legal requirement?"])

# replace(["Does the factory have a functioning Participation Committee?",
#          "Does the employer comply with requirements regarding shop stewards?	",
#          "Does the factory have a functioning bipartite cooperation institution?	",
#          "If Union Labour Committee is formed, does employer meet worker representatives on a regular basis?"])

# skipped welfare:

In [590]:
# replace([
#     "Adequate hand washing taps",
#     "Adequate washing facilities	"
# ],"Adequate washing facilities")

# replace([
#     "Does the employer comply with requirements regarding canteen?",
#     "Adequate eating area	","Does the workplace have adequate kitchen and dining areas?	"
# ])

# replace([
#     "Does the workplace have adequate changing rooms/lockers to meet the needs of workers?	",
#     "Does the workplace provide all workers a place to store their clothes?"
# ])

# replace([
#     "Does the workplace have all required facilities?rea?",
#     "Does the workplace have all required facilities?"
# ])

# replace([
#   "Does the workplace provide all workers a place to store their clothes?	",
#     "Does the workplace have all required facilities?",
#     "Does the workplace have adequate shelters or rest rooms?","Does the workplace have a functioning and accessible nursing room?"
# ],"locker, rest room or other required facilities")

# replace([
#     "Does the workplace have a functioning and accessible nursing room?","locker, rest room or other required facilities"
# ],"changing, nursing, rest room or other required facilities")

In [591]:
# 15
tab("Welfare Facilities")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Adequate eating area,584,0,161,833,474,119,1802,3973
Adequate washing facilities,584,2503,161,833,474,119,1802,6476
Does employer provide cups or other sanitary means\nfor drinking water?,0,2368,0,0,0,0,0,2368
Does the workplace have adequate accessible toilets?,584,2279,161,833,474,119,1802,6252
Does the workplace have adequate day care facilities?,584,1230,0,0,474,0,0,2288
Providing drinking water,584,2503,161,833,474,119,1802,6476
"When provided as in-kind payment, does the employer give workers enough food of decent quality?",0,0,0,0,474,0,0,474
"changing, nursing, rest room or other required facilities",584,2383,161,833,474,117,1802,6354
All,3504,13266,805,4165,3318,593,9010,34661


---
# new clean:

In [49]:
# replace(["Has the employer tried to prevent workers from participating in a strike?",
#          "Has the employer tried to prevent any workers from participating in a strike?"],"prevent strike participation")

In [283]:
# 15
tab("Strikes")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Has the employer hired new workers to replace striking workers during a strike?,584,2236,161,833,474,119,1802,6209
Punish workers for striking,584,2503,161,833,474,119,1802,6476
prevent strike participation,584,2236,161,833,474,119,1802,6209
All,1752,6975,483,2499,1422,357,5406,18894


In [282]:
# discard("Has the employer called security guards, the police or armed forces to break up a peaceful strike or arrest striking workers?")

In [484]:
#16:
tab("Collective Bargaining")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Are the provisions of the collective agreement at least as favourable for workers as the law?,584,2503,0,833,0,117,1802,5839
Are workers informed about the CBA?,0,2236,0,833,474,0,1802,5345
Does the employer consult with unions where legally required?,584,2503,156,833,474,119,1802,6471
Has the collective agreement in force been approved by more than 50% of workers covered?,0,0,0,0,0,0,1802,1802
Implement CBA,584,2503,161,833,474,119,1802,6476
Is the grassroots level union in the factory involved in the bargaining process at the enterprise level?,0,0,0,0,0,0,1802,1802
delete,584,0,0,833,0,0,1802,3219
All,2336,9745,317,4165,1422,355,12614,30954


~~[Does the employer refuse to bargain collectively or refuse to bargain in good faith with the union, worker representatives, union federations or confederations?	, [Haiti,Jordan]], other:drop~~

In [36]:
# replace([
#     "Does the employer consult with unions where legally required?	",
#     "Does the employer consult with worker representatives where legally required?"
# ])

# replace([
#     "Are workers informed about the CBA?",
#     "Does the employer provide workers access to the collective bargaining agreements?",
# "Has the employer made the collective bargaining agreement publically available to all workers?",
# "Does the employer inform workers about the contents of the collective bargaining agreement, and provide workers the text of the agreement?	"
# ])

# df.loc[(df.Country=="Haiti")&(df.QCleaned=="Does the employer refuse to bargain collectively or refuse to bargain in good faith with the union, worker representatives, union federations or confederations?"),"QCleaned"]\
#     ="Does the employer consult with unions where legally required?"

# df.loc[(df.Country=="Jordan")&(df.QCleaned=="Does the employer refuse to bargain collectively or refuse to bargain in good faith with the union, worker representatives, union federations or confederations?"),"QCleaned"]\
#     ="Does the employer consult with unions where legally required?"

In [499]:
#17:
tab("Regular Hours")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Do regular daily working hours exceed 10 hours?,584,2503,156,833,474,117,1802,6469
Do the attendance records reflect the hours actually worked?,584,2260,129,833,474,119,1802,6201
Does the employer comply with daily break periods?,584,0,161,0,0,119,1802,2666
one day off per week,584,2243,161,0,474,119,1802,5383
All,2336,7006,607,1666,1422,474,7208,20719


# *breast-feeding* solution:
- check `CPCleaned=leave`

In [498]:
df.loc[(df.CPCleaned=="Regular Hours")&(df.QCleaned=="provide break for pregnant worker or breast-feeding"),"CPCleaned"]="Leave"

In [293]:
# replace([
#     "Do workers have at least one day off per week?	",
#     "Does the employer give workers at least one day off per week?	",
#     "Does the employer provide required weekly rest periods?	",
#     "Does the employer provide a weekly rest day after six consecutive days of work?	"
# ],"one day off per week")

In [44]:
# replace([
#     "Do the attendance records reflect the hours actually worked?",
#     "Do the working time records reflect the hours actually worked?	"
# ])

# replace([
#     "Does the employer give workers at least one day off per week?	",
#     "Does the employer give workers at least one day off after 48 hours of work ?"
# ])

In [68]:
# replace([
#     "Does the employer collect and forward workers contributions to OFATMA?",
#     "Does the employer collect and forward workers' contributions to social insurance funds on time?	"
# ])

# v dont run this v:
# replace([
#     "Has the employer properly established a Workers' Participation Fund and Welfare Fund, and paid the required amount into the funds each year?",
#     "Does the employer pay the required amount each year into the Workers' Participation Fund?	",
#     "Does the employer pay the required employer contribution to the National Social Security Fund?",
#     "If a Provident Fund has been established, does the employer collect and deposit workers' contributions into the fund?"
# ], "pay into fund")

replace([
    "Does the employer provide nutritious food and drinks to female workers who work between 11 p.m. and 7 a.m?",
    "Does the employer provide round trip transport for female workers who work between 11 p.m. and 5 a.m?",
    "Does the employer provide transport home or a place to sleep for workers who finish work between 22:00 and 05:00?"
], "security or benefit after work")

# **!**

In [304]:
replace([
    "Are the Welfare Fund and Workers’ Welfare Foundation Fund used in the manner determined by the Fund Board of Trustees?	",
    "Are the Workers' Participation and Welfare Funds used and distributed as required?	"
])
# Has the employer properly established a Workers' Participation Fund and Welfare Fund, and paid the required amount into the funds each year?

In [356]:
# CPVC(18)
tab("Social Security and Other Benefits")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Are the Workers' Participation and Welfare Funds used and distributed as required?,584,0,0,0,0,0,0,584
Do workers receive required distributions from the Workers' Participation Fund?,18,0,0,0,0,0,0,18
Does the employer collect and forward workers contributions to OFATMA?,0,0,156,0,0,117,1802,2075
Does the employer comply with requirements when workers have occupational accidents and diseases?,0,0,0,0,0,0,1802,1802
Does the employer correctly calculate and send the employer contribution to the Social Security Corporation?,0,0,0,0,474,0,0,474
"Does the employer correctly calculate, collect, and forward workers' contributions to the Social Security Corporation?",0,0,0,0,474,0,0,474
Does the employer forward workers contributions to ONA?,0,0,161,0,0,0,0,161
Does the employer pay 2% of the gross payroll to INATEC?,0,0,0,0,0,119,0,119
Does the employer pay 3% of workers' basic salary to OFATMA for maternity and health insurance?,0,0,161,0,0,0,0,161


In [314]:
# replace([
#     "Does the employer pay workers two festival bonuses per year?	",
#     "Does the employer comply with other wage payments?	"
# ],"bonus or other wage")

# replace([
#     "Does the employer pay the required amount each year into the Welfare Fund and Workers’ Welfare Foundation Fund?",
#     "Has the employer properly established a Workers' Participation Fund and Welfare Fund, and paid the required amount into the funds each year?"
# ])

# replace([
#     "Does the employer pay all workers who work regularly the correct attendance bonus during leave legally required by law?	",
#     "Does the employer pay all workers who work regularly the correct attendance bonus when workers take annual leave?	"
# ])

# replace([
#     "Does the employer pay the required contributions for social insurance (BPJS Ketenagakerjaan) and health care funds (BPJS Kesehatan) to BPJS?",
#     "Does the employer comply with all requirements related to National healthcare funds (BPJS kesehatan), including registering workers, paying employer contributions, and collecting and forwarding workers’ contribution?"
# ],"pay social insurance")

# replace([
#     "Does the employer collect and forward all workers' contributions for social insurance (BPJS Ketenagakerjaan) and health care funds (BPJS Kesehatan)?",
#     "Does the employer comply with all requirements related to national social insurance (BPJS Ketenagakerjaan), including registering workers, paying employer contributions, and collecting and forwarding workers' contributions?"
# ],"forward social insurance")

---

In [504]:
# CPVC(19)
tab("Employment Contracts")
# df.loc[df.Q=="Do the employment contracts specify the terms and conditions of employment relation?","QCleaned"]="contract comply with local law"
# df.loc[df.Q=='Do the company regulations comply with legal requirements as stipulated in the labour law and regulations?',"QCleaned"]='Do the company regulations comply with legal requirements as stipulated in the labour law and regulations?'

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Do the employment contracts comply with other legal requirements?,584,2340,161,833,474,117,1802,6311
Do workers understand the terms and conditions of employment?,584,2330,161,0,474,117,1802,5468
company rules comply with local rules,0,2265,161,833,474,119,1802,5654
contract comply with local law,584,2286,161,833,474,119,1802,6259
provide all workers with appointment letter or copy,584,2503,161,833,474,0,1802,6357
All,2336,11724,805,3332,2370,472,9010,30049


### ~~[Do all persons who perform work for the factory, both on the premises and offsite, have a contract?, -Jordan]+provide all workers with appointment letter or copy~~

In [103]:
# replace([
#     "Has the employer provided all workers with a letter of appointment?",
#     "Does the employer provide workers with a copy of their contract?",
#     "Does the employer give a copy of the work agreement in Bahasa to workers?"
# ],"provide all workers with appointment letter or copy")

# replace([
#     "Do the employment contracts comply with other legal requirements?	",
#     "Do the letters of appointment specify the terms and conditions of employment?	"
# ])

# replace([
#     "Do the contracts comply with the labour law, collective agreement and work rules?",
#     "Do the employment contracts comply with Cambodian labour law and with the factory's internal regulations?",
#     "Do the employment contracts comply with Jordanian legal requirements?",
#     "Do the letters of appointment comply with the labour law, collective agreement and service rules?"
# ],"contract comply with local law")

# replace([
#     "Do the company regulations comply with legal requirements as stipulated in the labour law and regulations?",
#     "Do the internal work rules comply with national law?"
# ],"company rules comply with local rules")

# replace(["company rules comply with local rules",
#          "Do the factory bylaws comply with Jordanian legal requirements and were they communicated to workers?	"])

# replace([
#     "Does the employer comply with limits on the use of work agreements for a specified period of time (i.e., limits on the employment of non-permanent workers)?",
#     "Does the employer comply with limits on the period of employment for training workers?",
#     "Is the probationary period limited to 3 months?"
# ], "limited probationary period")

# df.loc[(df.QCleaned=="Do all persons who perform work for the factory, both on the premises and offsite, have a contract?")\
#        &(df.Country=="Jordan"), "CPCleaned"]="delete"

# replace([
#     "Do all persons who perform work for the factory, both on the premises and offsite, have a contract?",
#     "provide all workers with appointment letter or copy"
# ])

---
# 20: no Haiti? (combined with 19?)

In [382]:
# CPVC(20)
tab("Contracting Procedures")

Country,Bangladesh,Cambodia,Indonesia,Jordan,Vietnam,All
QCleaned,,,,,,
"Do all workers (other than apprentices, badli or casual workers) have a service book?",584,0,0,0,0,584
Do the migrant workers have valid work permits and residence IDs?,0,0,0,474,0,474
Does the employer comply with legal requirements when subcontracting part of its work to another enterprise?,0,0,833,0,0,833
"Does the employer comply with limits on the use of work agreements for a specified period of time (i.e., limits on the employment of non-permanent workers)?",0,0,833,0,0,833
Does the employer comply with requirements for apprentices?,0,0,480,0,1802,2282
Does the employer comply with requirements for temporary transfers of workers to new work?,0,0,0,0,1802,1802
Does the employer comply with the legal limit on the use of fixed term contracts?,0,2503,0,0,0,2503
"Does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?",0,2503,0,0,0,2503
limited probationary period,584,2241,833,474,1802,5934


In [381]:
# replace([
#     "Does the employer comply with legal requirements relating to outsourced workers?	",
#     "Does the employer comply with requirements concerning sub-contracted workers at the workplace?",
#     "Have you found non-compliance with legal requirements for compensation, contracts, OSH and/or working time pertaining to non-production workers and/or sub-contracted workers?	"
# ],"requirement on sub-contracted of outsourced workers")

# replace([
#     "Does the employer comply with national law concerning dispatched workers at the workplace?",
#     "requirement on sub-contracted of outsourced workers"
# ])

# replace([
#     "Do workers have to pay anyone to get a job?",
#     "Do workers pay any recruitment fees?"
# ],"workers pay for job")

# **!**

In [ ]:
# df.loc[df.Q=="Does the employer dismiss workers for reasons that are not allowed by law?","QCleaned"]="Does the employer dismiss workers for reasons that are not allowed by law?"

In [408]:
# CPVC(21)
# df.loc[df.CPCleaned=="Termination","QCleaned"]=df[df.CPCleaned=="Termination"].Q
tab("Termination")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Does the employer compensate workers for unused paid annual leave when they resign or are terminated?,584,2503,161,833,474,117,1802,6474
Does the employer only terminate workers for valid reasons?,584,2503,161,833,474,0,1801,6356
comply legal requirements before terminating workers,566,2503,156,0,0,117,1802,5144
comply severance pay,584,2503,0,833,0,0,1802,5722
give workers chance to defend for termination,584,0,0,833,474,0,1802,3693
notice worker before dismissal,584,2503,161,0,474,0,1802,5524
resigned or terminated workers receive correct wage,584,2412,161,0,474,0,1802,5433
All,4070,14927,800,3332,2370,234,12613,38346


In [407]:
# func for cp=deletion
df.loc[df.QCleaned=="Does the employer pay terminated workers the correct damages when required?","CPCleaned"]="delete"

In [405]:
# discard("Does the employer settle all the payments within seven working days after termination of the labour contract?")
# discard("Does the employer pay all outstanding wages and indemnities within 48 hours of termination?")
# discard("Does the employer unilaterally terminate workers for reasons that are not allowed by law?")

In [385]:
replace([
    "Does the employer only terminate workers for valid reasons?	",
    "Does the employer terminate workers for reasons that are invalid under national law?	",
    "Does the employer dismiss workers for reasons that are not allowed by law?"
])

In [151]:
# replace([
#     "Does the employer compensate workers for unused paid annual leave and other legally required termination/resignation benefits?",
#     "Does the employer compensate workers for unused paid annual leave and unused earned compensatory rest day/s when they resign or are terminated",
#     "Does the employer compensate workers for unused paid annual leave when they resign or are terminated?"
#     ,"Do workers who resign or are terminated receive severance based on years of service, unused annual leave, the accumulated 13th month payment, and any other legally required benefits?"
#     ,"Does the employer compensate workers for unused paid annual leave and unused earned compensatory rest day/s when they resign or are terminated?"])

# replace([
#     "Does the employer only terminate or dismiss workers for valid reasons?",
#     "Does the employer only terminate workers for valid reasons?",
# ])


# replace(
#     ["Are workers given the opportunities required under law to defend themselves before they are terminated?	",
# "Do workers have an opportunity to defend themselves before they are dismissed based on their conduct or performance?",
# "Do workers have an opportunity to defend themselves before they are dismissed or punished based on their conduct or performance?",
# "Do workers have an opportunity to defend themselves before they are terminated based on their conduct or performance?"]
# ,"give workers chance to defend for termination")

# replace([
#     "Do resigned or terminated workers receive all other termination benefits required by law?",
# "Do resigned or terminated workers receive correct wage for all days worked and other termination benefits required by law?	"
# ],"resigned or terminated workers receive correct wage")

# replace([
#     "Does the employer comply with legal requirements before laying off workers due to force majeure, and before retrenching workers due to redundancy?	",
# "Does the employer comply with legal requirements before reducing the size of the workforce or suspending workers' contracts due to changes in structure, technology or economic reasons?",
# "Does the employer comply with legal requirements before suspending workers or reducing the size of the workforce due to changes in operations, lack of materials, force majeure, or accident resulting in an immediate work stoppage?",
# "Does the employer comply with legal requirements before suspending workers or reducing the size of the workforce due to changes in operations?"
# ], "comply legal requirements before terminating workers")

# replace(
#     ["Does the employer comply with requirements regarding severance pay and job-loss allowance?",
#      "Does the employer comply with requirements regarding severance pay and reward for service?",
#      "Does the employer comply with requirements regarding severance pay?"],"comply severance pay"
# )

# replace([
#     "Does the employer pay workers their outstanding wages and the correct indemnity for dismissal?	",
#     "Does the employer pay workers their outstanding wages and the correct seniority indemnity for dismissal?"
# ])

# replace(["Does the employer provide terminated workers their Provident Funds (if applicable)?	",
#         "Does the employer provide workers their Provident Funds when they resign or are terminated? (if applicable)"])

replace(["Does the employer provide workers proper notice of termination when required, or pay workers during the notice period?",
"Does the employer provide workers proper notice of termination?",
"Does the employer provide workers proper notice, and an opportunity to defend themselves before they are terminated based on their conduct or performance?"],
"notice worker before dismissal")

In [511]:
# CPVC(22)
# df.loc[df.Q=="Are pregnant workers provided with time off for prenatal visits?","QCleaned"]="Are pregnant workers provided with time off for prenatal visits?"
# df.loc[df.Q=="Are pregnant workers provided time off for prenatal visits?","QCleaned"]="Are pregnant workers provided time off for prenatal visits?"
tab("Leave")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Are pregnant workers provided time off for prenatal visits?,0,0,0,0,0,0,1802,1802
Can female workers take time off when they feel pain during the first two days of menstruation and they notify the employer?,0,0,0,833,0,0,0,833
Does the employer allow workers to take 30 minutes rest during their period?,0,0,0,0,0,0,1802,1802
Does the employer allow workers to take time off for paternity/ personal or family matters as required by national law?,0,0,0,833,0,0,0,833
Does the employer comply with the entitlement to paternity leave?,0,0,0,0,0,0,1802,1802
Does the employer give workers up to 7 days of time off for special leave per year?,0,2503,0,0,0,0,0,2503
Does the employer provide 12 days of annual leave per year?,584,2503,161,833,474,0,1802,6357
Does the employer provide 14 days of sick leave per year?,584,2503,0,833,474,0,1802,6196
Does the employer provide required annual leave?,584,0,0,0,0,0,0,584


### ~~drop~~ (done):
- Does the employer provide 14 days of sick leave per year?	, haiti
- maternity or paternity leave	,haiti
- provide break for pregnant worker or breast-feeding, haiti

In [185]:
# df.loc[(df.Country=="Haiti")&\
#    (df.QCleaned=="provide break for pregnant worker or breast-feeding"),"CPCleaned"]="delete"

# replace([
#     "Does the employer provide workers 11 festival holidays per year (or compensatory and substitute holidays when workers work on festival holidays)?	",
#     "Does the employer provide workers at least 11 festival holidays per year (or compensatory and substitute holidays when workers work on festival holidays)?	"
# ], "provide minimal 11 holidays per year")

# replace([
#     "Does the employer provide required breastfeeding breaks?",
#     "Does the employer provide required time off for breastfeeding breaks?	"
# ], "provide break for breastfeeding")

# replace([
#     "Are pregnant workers provided time off for prenatal visits?",
# "Are pregnant workers provided with time off for prenatal visits?"	], "provide break for pregnant workers")

# replace([
#      "provide break for pregnant worker or breast-feeding",
#      "Does the employer provide opportunities for breastfeeding breaks?"])

# replace(["Does the employer comply with the entitlement to sick leave and other types of leave that are covered by social insurance?",
#          "Does the employer provide 14 days of sick leave per year?"])

# df.loc[(df.QCleaned=="Does the employer provide required annual leave?") & (df.Country=="Vietnam"),"QCleaned"]="Does the employer provide 12 days of annual leave per year?"

# replace([
#     "Does the employer provide required time off for paternity leave?",
#     "Does the employer provide at least 12 weeks of maternity leave?"
# ], "maternity or paternity leave")

In [430]:
#23:
# CPVC(23)
tab("Union Operations")


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Do union representatives have access to workers in the workplace?,584,2503,161,833,474,119,1802,6476
Does the employer deduct union dues from wages when workers request this in writing?,584,2240,161,833,463,119,1775,6175
provide facilities for unions,584,2503,0,833,474,0,1802,6196
All,1752,7246,322,2499,1411,238,5379,18847


In [429]:
# replace(
#     ["Are workers effectively trained and encouraged to properly use the personal protective equipment and machines?	",
#  "Are workers effectively trained and encouraged to properly use personal protective equipment and machines?	",
#  "Are workers effectively trained and encouraged to properly use personal protective equipment, machines and equipment?	",
#  "Are workers effectively trained and encouraged to use the personal protective equipment that is provided?	",
#  "Are workers effectively trained on occupational health and safety?	",
#  'Are workers effectively trained and encouraged  to properly  use the personal protective equipment  and machines?',
#  "Are workers effectively trained to use the personal protective equipment and machines?","Are all workers trained and obligated to use machines and equipment safely?"]
# ,"training for protection")

# replace([
#     "Do workers have suitable chairs?",
#     "Does the employer comply with ergonomic requirements?	"
# ])

# replace([
#     "Are workers punished if they remove themselves from work situations that they believe present an imminent and serious danger to life or health?",
# "Does the employer force workers to continue working when they have refused to work due to clear imminent and serious danger to their life or health?"
# ],"prevent worker from leaving dangerous workplace")

# replace([
# "provide facilities for unions",
# "Does the employer provide the Collective Bargaining Agent with an office at the workplace?	"])


In [440]:
#24:
# CPVC(24)
tab("Worker Protection")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Are all workers trained and obliged to use the personal protective equipment that is provided?,0,0,0,833,0,0,0,833
Comply with machines and equipment requirements,0,0,0,0,0,0,1802,1802
Do workers have suitable chairs?,584,2321,161,833,474,119,1802,6294
Do workers use the machines and equipment in a safe manner?,0,0,0,0,0,0,1802,1802
Do workers use the personal protective equipment that is provided?,0,0,0,0,0,0,1802,1802
Electrical wires/switches/plugs properly installed,584,2503,161,833,474,119,1802,6476
Proper guards installed and maintained,584,2503,161,833,474,119,1802,6476
Protect workers from falls from heights,357,2241,0,0,0,0,1802,4400
Provide PPE,584,2503,161,833,474,119,1802,6476


In [448]:
# for i in ["Has the employer provided workers who work with chemical substances with proper clothing and personal protective equipment when necessary?",
# "Has the employer provided workers with dust masks when necessary?",
# "Has the employer provided workers with ear plugs when necessary?",
# "Has the employer provided workers with metal gloves when necessary?",
# "Has the employer provided workers with other PPE when necessary?"]:
#     discard(i)

# discard("Boilers registered/certified")

# replace(["Comply with strict occupational safety requirement","personnel has related legal permits"])

# df.loc[df.QCleaned=="Has the factory conducted an assessment of the potential hazards in the workplace?","CPCleaned"]="delete"
# discard("Is the workplace overcrowded?")

In [464]:


df.loc[df.Q.str.contains("Do workers have a medical check within the first three months of hiring ?"),"CPCleaned"]="delete"
# df.loc[df.Q=="Do workers receive the health cards within the first 3 months of hiring ?","CPCleaned"]


In [538]:
#26:
# CPVC(26)
tab("Health Services and First Aid")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with HIV/AIDS policy,584,2503,0,833,0,119,1802,5841
Do workers who have been exposed to work-related hazards receive free health checks?,0,0,161,0,0,0,0,161
Does the employer address safety and health risks to pregnant or nursing workers?,584,2236,161,833,0,119,1802,5735
Does the employer provide required medical services?,584,2503,161,833,474,0,1802,6357
Trained workers on first aid,584,2286,161,833,474,119,1802,6259
enough first aid boxes,584,2271,161,833,474,119,1802,6244
workers get health check or report,584,2386,155,833,474,117,1802,6351
All,3504,14185,960,4998,1896,593,10812,36948


# ~~**recover**~~ (done)

In [257]:
# replace(["Has the employer ensured that there are a sufficient number of readily accessible first aid boxes/ supplies in the workplace?",
# "Has the employer ensured that there are a sufficient number of readily accessible first aid boxes/supplies in the workplace?",
# "Has the employer ensured that there is a sufficient number of readily accessible first aid boxes/supplies in the workplace?",
# "Has the employer ensured there are a sufficient number of readily accessible first aid boxes/supplies in the workplace?",],
# "enough first aid boxes")


# replace([
# "Do workers have annual medical checks?",
# "Do workers receive free periodic health checks as required by law?",
# "Do workers receive the health cards within the first 3 months of hiring ?",
# "Do workers undergo a medical examination before being employed?"
# ], "workers get health check or report")

# replace([
#     "Does the workplace have sufficient onsite medical facilities and staff?",
#     "Does the workplace have required onsite medical facilities and staff?"
# ,"Does the employer provide required medical services?	"
# ,"Does the workplace have an adequately equipped and staffed infirmary?"
# ,"Does the workplace have required onsite medical facilities and staff?	"
# ])

In [261]:
#27-OK?:
# CPVC(27)
tab("Hazardous Work and other Worst Forms")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Vietnam,All
QCleaned,,,,,,,
Do workers who are under age 18 perform work that is hazardous by nature?,584,2236,161,833,0,1802,5616
Do workers who are under age 18 work overtime or work at night?,584,2236,0,833,474,1802,5929
All,1168,4472,161,1666,474,3604,11545


# __?-only vietnam__

In [263]:
#28:
# CPVC(28)
tab("Religion and Political Opinion")

Country,Vietnam,All
QCleaned,,
Are workers harassed on the basis of their religion or political opinion?,1802,1802
Have you found that religion or political opinion is a factor in the employer's decisions regarding termination or retirement of workers?,1802,1802
All,3604,3604


In [470]:
#29-OK:
# CPVC(29)
tab("Method of Payment")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Are wages paid on time?,584,2294,161,833,474,119,1802,6267
All,584,2294,161,833,474,119,1802,6267


In [270]:
# replace([
#     "Are wages paid on time?	","Are workers paid on time, twice per month?	",
#     "Are workers paid within seven days from the date wages become payable?",
#     "Does the employer pay wages on time and pay interest in case of late payment?"
# ])

# replace([
#     "Do in-kind wage payments comply with workers' employment contracts?	",
#     "Are workers' full wages paid directly to workers at the workplace on working days?"
# ], "direct or in-kind payment")

# df.loc[df.QCleaned=="direct or in-kind payment", "CPCleaned"]="delete"

# **!** &rarr; prof:check

In [475]:
#30:
# CPVC(30)
tab("Worker Accommodation")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Vietnam,All
QCleaned,,,,,,,
"Adequate toilets, shower, sewage in dorm",584,3,161,833,474,1802,3857
Does the accommodation comply with minimum space requirements?,584,0,161,833,474,0,2052
Does the accommodation have adequate cooking facilities?,584,3,161,833,474,0,2055
Does the accommodation have adequate eating and living areas?,0,0,0,0,474,0,474
Does the accommodation have enough safe water?,584,3,161,833,474,0,2055
Does the accommodation have laundry facilities?,0,0,0,0,474,0,474
Does the accommodation offer workers adequate privacy?,584,3,161,833,474,1802,3857
Has the employer adequately prepared for emergencies in the accommodation?,584,2271,161,833,474,1802,6125
Is the accommodation adequately lit?,584,3,161,833,474,0,2055


In [278]:
# replace([
#     "Is the accommodation adequately protected against heat, cold, dampness, and noise, and is it adequately ventilated and lit?",
#     "Is the accommodation adequately lit?"
# ])

# replace([
#     "Are trained first aid personnel and adequate first aid supplies available in the accommodation?",
#     "Has the employer adequately prepared for emergencies in the accommodation?"
# ])


In [542]:
#31:
# CPVC(31)
tab("Race and Origin")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"Do recruitment materials such as job announcements or job application forms refer to the applicant's race, colour or origin?",584,2503,161,0,0,119,1802,5169
"Is an applicant's race, colour or origin a factor in hiring decisions?",584,2503,161,0,0,119,1802,5169
"Is applicant's race, color or origin a factor in recruitment or hiring?",0,0,0,833,0,0,0,833
"Is race, colour or origin a factor in decisions regarding opportunities for promotion or access to training?",584,2503,161,0,474,119,1802,5643
"Is race, colour, origin or citizenship status a factor in decisions regarding pay?",0,0,0,0,474,0,0,474
All,1752,7509,483,833,948,357,5406,17288


In [544]:
#32:
# CPVC(32)
tab("Other Grounds")

Country,Indonesia,Jordan,Vietnam,All
QCleaned,,,,
Do recruitment materials such as job announcements or job application forms refer to age in a manner that could discourage older workers from applying?,0,0,721,721
Does the employer hire one disabled worker for every 100 workers?,833,0,0,833
Has the employer made required accommodations for physically disabled workers?,833,474,1802,3109
Is age a factor in decisions regarding opportunities for promotion or access to training?,0,0,721,721
Is an applicant's age a factor in hiring decisions?,0,0,721,721
All,1666,474,3965,6105


In [564]:
#33:
# CPVC(33)
tab("Overtime")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Does the employer ensure that workers have on average at least 4 rest days per month when weekly rest is not possible?,0,0,0,0,0,0,1802,1802
Does the employer get permission from the Labor Inspector before suspending weekly rest days?,0,2276,0,0,0,0,0,2276
Does the employer get written consent from women to work at night?,584,0,0,0,0,0,0,584
Does the employer prepare written instructions on overtime?,584,0,0,833,0,0,0,1417
Does the employer provide compensatory days off when workers work on weekly rest days?,584,0,0,0,0,0,0,584
Is overtime voluntary?,584,2503,161,833,474,119,1802,6476
comply daily overtime limit,584,2503,101,833,0,119,1802,5942
comply monthly overtime limit,0,0,0,0,0,0,1802,1802
comply yearly overtime limit,0,0,0,0,0,0,1802,1802


[e.g. of problem:](.\prob.png)
# ??:
`Does the employer prepare written instructions on overtime?`+`get permission for letting workers work overtime` 

In [560]:
df[(df.Country=="Haiti")&(df.Q.str.contains("overtime"))].Q.value_counts()

Is overtime work voluntary?                                                                                                                                             161
Are the emergency exits accessible, unobstructed and unlocked during working hours, including overtime? [differentiation question]                                      106
Does the employer pay workers 50% above the normal wage for all ordinary overtime hours worked? [differentiation question]                                              106
Are workers forced to work overtime under threat of penalty? [differentiation question]                                                                                 106
Does the employer comply with limits on overtime hours worked?                                                                                                          101
Does the employer obtain authorization from the Department of Labour before working overtime?                                               

In [561]:
# replace([
#     "Does the employer obtain authorization from the Department of Labour before working on Sundays?",
#     "Does the employer get permission from the Labour Inspector before workers work overtime?"
# ],"permit for letting workers work overtime")


# replace([
#     "Does the employer comply with monthly limits on overtime hours worked?	",
# "Does the employer comply with the monthly limits on overtime hours worked?	"
# ],"comply monthly overtime limit")

# replace([
#     "Does the employer comply with yearly limits on overtime hours worked?",
# "Does the employer comply with yearly  limits on overtime hours worked?"
# ],"comply yearly overtime limit")

# replace([
#     "Does the employer comply with daily limits on overtime hours worked?	",
#     "Does the employer comply with limits on overtime hours worked?	",
#     "Is overtime limited to 3 hours per day, 14 hours per week?",
#     "Is overtime limited to three hours per day and nine hours per week?	",
#     "Is overtime work limited to two hours per day?"
# ],"comply daily overtime limit")


In [570]:
#34-OK:
# CPVC(34)
tab("Documentation and Protection of Young Workers")

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Vietnam,All
QCleaned,,,,,,,
Does the employer comply with requirements regarding the certificate of fitness for workers under age 18?,584,0,0,0,0,0,584
Does the employer keep a record of workers under 18 years of age?,584,2237,161,833,474,1802,6091
Does the employer reliably verify the age of workers prior to hiring?,584,2503,161,833,0,1802,5883
All,1752,4740,322,1666,474,3604,12558


In [569]:
# replace([
#     "Does the employer keep a record of workers under 18 years of age?	",
#     "Does the employer comply with documentation requirements for workers under age of 18?"
# ])

# replace([
#     "Does the employer have a verification system in place to ensure they do not hire child labour in accordance with the National Law?	",
#     "Does the employer reliably verify the age of workers prior to hiring?	"
# ])

In [572]:
df.to_csv("r2.9.csv",index=False)

In [596]:
df.to_stata("cleaned.dta", version=118)

In [595]:
df[df.Q.str.contains("\u2019")].Q.unique()

array(['Does the employer provide 1.5 months of maternity rest before a worker’s estimated due date and another 1.5 months after delivery?',
       "Does the employer provide paid sick leave as required by the factory’s Internal Regulations, or if there are no Internal Regulations, according to the Ministry's practice?",
       'Does the employer pay the required amount each year into the Welfare Fund and Workers’ Welfare Foundation Fund?',
       'Are the Welfare Fund and Workers’ Welfare Foundation Fund used in the manner determined by the Fund Board of Trustees?',
       'Does the employer comply with all requirements related to National healthcare funds (BPJS kesehatan), including registering workers, paying employer contributions, and collecting and forwarding workers’ contribution?'],
      dtype=object)

<center>
<h1>---Finish---</h1>
</center>

In [598]:
df["idx"]=df.index